# scikit‑learn: Linear & Polynomial Regression using cross validation

This notebook extends on the simple linear regression notebook:
- **Linear regression** on a simple dataset (A)  
- **Hold‑out train/test split** to test generalization (test set never used during model selection)  
- **Cross‑validation (CV)** to compare models **on the training set** only  
- **Polynomial regression** on dataset (B) using a `Pipeline(PolynomialFeatures → StandardScaler → Ridge)`

**Data files (already included):**
- `data/dataset_a.csv` — roughly linear  
- `data/dataset_b.csv` — needs polynomial features

> Tip: keep this notebook and the `data/` folder in the same directory.


In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import matplotlib.pyplot as plt

DATA_DIR = Path("data")

def load_xy(csv_path):
    df = pd.read_csv(csv_path)
    X = df[['x']].values  # shape (n, 1)
    y = df['y'].values
    return df, X, y

def report_metrics(name, y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    r2  = r2_score(y_true, y_pred)
    print(f"{name:>10} | MSE = {mse:.3f}, MAE = {mae:.3f}, R^2 = {r2:.3f}")
    return dict(mse=mse, mae=mae, r2=r2)

def plot_fit(ax, X, y, model, title):
    ax.scatter(X.ravel(), y, alpha=0.8)
    xs = np.linspace(X.min()-0.5, X.max()+0.5, 300).reshape(-1, 1)
    ys = model.predict(xs)
    ax.plot(xs, ys)
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_title(title)
    return ax


## Part 1 — Dataset A (Linear)

1. Split the data into **train** and **test** (e.g., 80/20).  
2. Fit `LinearRegression` on the **training** data.  
3. Use **5‑fold CV** (on the training set) to estimate training‑time performance.  
4. Evaluate the single final model on the **held‑out test** set to estimate generalization.


In [ ]:
# Load dataset A
df_a, X_a, y_a = load_xy(DATA_DIR / 'dataset_a.csv')
df_a.head()


In [ ]:
# 1) Train/test split
Xtr, Xte, ytr, yte = train_test_split(X_a, y_a, test_size=0.2, random_state=42)

# 2) Fit Linear Regression
lin_reg = LinearRegression()
lin_reg.fit(Xtr, ytr)

# 3) Cross‑validation on the *training* split only
cv_scores = cross_val_score(lin_reg, Xtr, ytr, scoring='neg_mean_squared_error', cv=5)
cv_rmse = (-cv_scores) ** 0.5
print(f"CV RMSE (mean ± std): {cv_rmse.mean():.3f} ± {cv_rmse.std():.3f}")

# 4) Evaluate on train and held‑out test
yhat_tr = lin_reg.predict(Xtr)
yhat_te = lin_reg.predict(Xte)

train_metrics = report_metrics("train", ytr, yhat_tr)
test_metrics  = report_metrics("   test", yte, yhat_te)


In [ ]:
# Plot fit on full data for visualization
fig, ax = plt.subplots(figsize=(6,4))
plot_fit(ax, X_a, y_a, lin_reg, "Dataset A — Linear Regression fit")
plt.show()


**Connect to lecture slides:**  
- Multiple linear regression notation and vectorization (Lecture 5).  
- We'll keep one feature here for clarity and match the straight‑line model.

Later we’ll mirror the **Polynomial Regression** slides and include **feature scaling** for stability.


### Why both CV *and* a held‑out test set?

- Use **K‑fold Cross‑Validation on the *training*** data to choose/compare models (e.g., degree of polynomial, regularization).  
- Keep a **separate test set**, untouched during CV/model selection, to evaluate **generalization** once at the end.


## Part 2 — Dataset B (Polynomial features + regularization)

We’ll search over polynomial **degree** and Ridge **alpha** using CV on the training split,
then report performance on the held‑out test set.


In [ ]:
# Load dataset B
df_b, X_b, y_b = load_xy(DATA_DIR / 'dataset_b.csv')
df_b.head()


In [ ]:
# Train/test split
Xtr_b, Xte_b, ytr_b, yte_b = train_test_split(X_b, y_b, test_size=0.2, random_state=42)

# Build a pipeline: PolynomialFeatures -> StandardScaler -> Ridge
pipe = Pipeline([
    ('poly', PolynomialFeatures(include_bias=False)),
    ('scaler', StandardScaler()),
    ('reg', Ridge())
])

# Hyperparameter grid
from sklearn.model_selection import GridSearchCV
param_grid = {
    'poly__degree': [1, 2, 3, 4, 5, 6],
    'reg__alpha': [0.0, 0.1, 1.0, 10.0]
}

gcv = GridSearchCV(
    pipe, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=None, refit=True
)
gcv.fit(Xtr_b, ytr_b)

print("Best params:", gcv.best_params_)
print("CV RMSE (best):", (-gcv.best_score_) ** 0.5)

best_model = gcv.best_estimator_

# Evaluate on train and held‑out test
yhat_tr_b = best_model.predict(Xtr_b)
yhat_te_b = best_model.predict(Xte_b)

train_metrics_b = report_metrics("train", ytr_b, yhat_tr_b)
test_metrics_b  = report_metrics("   test", yte_b, yhat_te_b)


In [ ]:
# Plot the best polynomial fit
fig, ax = plt.subplots(figsize=(6,4))
plot_fit(ax, X_b, y_b, gcv.best_estimator_, f"Dataset B — Best fit (degree={gcv.best_params_['poly__degree']}, alpha={gcv.best_params_['reg__alpha']})")
plt.show()


In [ ]:
# Visualize how CV error changes with degree (alpha fixed at the best alpha)
import numpy as np
results = pd.DataFrame(gcv.cv_results_)

# Find row with best alpha per degree (lowest mean test MSE)
summary = (results
           .assign(degree=results['param_poly__degree'].astype(int),
                   alpha=results['param_reg__alpha'].astype(float),
                   rmse=np.sqrt(-results['mean_test_score'].values))
           .sort_values(['degree','rmse'])
          )

best_alpha = gcv.best_params_['reg__alpha']

deg_grid = sorted(summary['degree'].unique())
deg_rmse = [summary.query('degree==@d and alpha==@best_alpha')['rmse'].min() for d in deg_grid]

fig, ax = plt.subplots(figsize=(6,4))
ax.plot(deg_grid, deg_rmse, marker='o')
ax.set_xlabel('Polynomial degree')
ax.set_ylabel('CV RMSE (alpha fixed at best)')
ax.set_title('Cross‑validated error vs. model complexity (training only)')
plt.show()
